In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd

from quick_pp.database.objects import Project
from quick_pp.database.db_connector import DBConnector

# Load well from saved file
project_name = "MOCK_clastic"
db_conn = DBConnector()
with db_conn.get_session() as db_session:
    project = Project(db_session, name=project_name)
    df = project.get_all_data()
    print(project.get_well_names())

merged_df = pd.DataFrame()
for well_name, well_data in df.groupby('WELL_NAME'):
    if well_name in ['15-9-19-A', '15-9-19-BT2']:
        core_data = pd.read_excel(rf"data\01_raw\VOLVE\{well_name}_RCA.xls", skiprows=3).iloc[1:]
        core_data['DEPTH'] = pd.to_numeric(core_data['Depth'], errors='coerce')
        core_data['CPORE'] = pd.to_numeric(core_data['Por., hor.'], errors='coerce') / 100
        core_data['CPERM'] = pd.to_numeric(core_data['Kl, hor'], errors='coerce')
        temp_df = pd.merge_asof(well_data, core_data[['DEPTH', 'CPORE', 'CPERM']],
                                direction='nearest', on='DEPTH', tolerance=0.1524 / 2)
        merged_df = pd.concat([merged_df, temp_df])

In [ ]:
# Load data
well_name = '15-9-19-A'
well_data = merged_df[merged_df.WELL_NAME == well_name]

# Quick PP Interpretation

In [ ]:
from quick_pp.lithology.sand_shale import SandShale
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix, normalize_volumetric, estimate_shale_porosity_trend
from quick_pp.qaqc import *
from quick_pp.saturation import *
from quick_pp.plotter.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import rock_typing, estimate_vsh_gr
from quick_pp.plotter.plotter import plotly_log
from quick_pp.utils import *

## Estimate Litholoy

In [ ]:
# Clean up data
well_data = badhole_flagging(well_data)
# well_data = handle_outer_limit(well_data, fill=True)

for col in ['GR', 'RT', 'NPHI', 'RHOB']:
    well_data.loc[:, col] = remove_straights(well_data[col])

# Initialize lithology model
args = {
    'litho_model': 'ss',
    # 'dry_clay_point': (.3, 2.7),
    'wet_clay_point': (0.45, 2.45),
    'sw_water_salinity': 15000,
    'sw_m': 1.85,
    'sw_n': 1.85,
    'hc_corr_angle': neu_den_xplot_hc_correction_angle(rho_water=1.0, rho_hc=0.8, HI_hc=0.9),
    'hc_buffer': 0.01,
    'ressum_cutoffs': dict(
        VSHALE=.5,
        PHIT=0,
        SWT=1
    ),
}
ss_model = SandShale(**args)
vsand, vcld, _ = ss_model.estimate_lithology(
    nphi=well_data['NPHI'], rhob=well_data['RHOB']
)
args.update(ss_model.__dict__)

# Implement hydrocarbon correction
vsh_gr = estimate_vsh_gr(well_data['GR'])
nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
    well_data['NPHI'], well_data['RHOB'], vsh_gr=vsh_gr,
    dry_min1_point=args['dry_sand_point'],
    dry_clay_point=args['dry_clay_point'],
    corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
)

# Correct density log
rhob_corr = den_correction(nphihc, well_data['GR'], vsh_gr=vsh_gr, alpha=0.1)
badhole_flag =  np.where(abs(well_data['RHOB'] - rhob_corr) > 0.2, 1, 0)
rhob_corr = np.where((badhole_flag == 1) & (hc_flag == 0), rhob_corr, rhobhc)

# Estimate lithology
ss_model = SandShale(**args)
vsand, vcld, (nphi_max_line, rhob_max_line) = ss_model.estimate_lithology(
    nphi=nphihc, rhob=rhob_corr,
)

# Update ZONES
well_data = zone_flagging(well_data)
zones = well_data.ZONES

In [ ]:
neutron_density_xplot(well_data['NPHI'], well_data['RHOB'], dry_min1_point=args['dry_sand_point'], **args)

In [ ]:
args['hc_corr_angle']

In [ ]:
neutron_density_xplot(nphihc, rhobhc, dry_min1_point=args['dry_sand_point'], **args)

In [ ]:
# Specify depth of interest
min_depth = well_data.DEPTH.min()
max_depth = well_data.DEPTH.max()

In [ ]:
# Density correction based on VSH_GR

plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, rhob_corr, label='rhob_corr')
plt.plot(well_data.DEPTH, well_data['RHOB'], label='RHOB')
plt.plot(well_data.DEPTH, rhobhc, label='RHOB_HC')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.ylim(1.5, 3)
plt.xlim(min_depth, max_depth)

plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, well_data['NPHI'], label='NPHI')
plt.plot(well_data.DEPTH, nphihc, label='NPHI_HC')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.ylim(0, .5)
plt.xlim(min_depth, max_depth)

plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, badhole_flag, label='badhole_flag')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.xlim(min_depth, max_depth)

## Estimate Porosity

In [ ]:
args

In [ ]:
# Estimate porosity
phit = neu_den_xplot_poro(
    nphihc, rhob_corr, model='ss',
    dry_min1_point=args['dry_sand_point'],
    dry_clay_point=args['dry_clay_point'],
)

rho_ma = rho_matrix(vsand, vcld)
phid = density_porosity(rhob_corr, rho_ma)

# Normalize lithology
volumes = dict(vcld=vcld, vsand=vsand)
volumes = normalize_volumetric(phit, **volumes)
vcld, vsand = volumes['vcld'], volumes['vsand']

# Calculate vclb: volume of clay bound water and phie
clay_phit = estimate_shale_porosity_trend(rhob_corr, args['dry_clay_point'][1])
vclb = vcld * clay_phit
vclay = vcld + vclb

phie = phit - vclb

# Estimate rock types
rock_flag = rock_typing(vclay, higher_is_better=False)

In [ ]:
from quick_pp.rock_type import estimate_vsh_gr
vsh_gr_1 = estimate_vsh_gr(well_data['GR'])

fig, axs = plt.subplots(2, 1, figsize=(20, 3), sharex=True)
axs[0].plot(well_data.DEPTH, vsh_gr_1, label='vsh_gr_1')
axs[0].plot(well_data.DEPTH, vclay, label='vclay')
axs[0].plot(well_data.DEPTH, vcld, label='vcld')
axs[0].set_ylim(-.1, 1.1)
axs[0].legend()

axs[1].plot(well_data.DEPTH, well_data['GR'], label='GR')
axs[1].set_ylim(0, 200)
axs[1].legend()

In [ ]:
plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, phit, label='PHIT')
plt.plot(well_data.DEPTH, phid, label='PHID')
plt.plot(well_data.DEPTH, phie, label='PHIE')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.xlim(min_depth, max_depth)
plt.ylim(0, 0.5)

## Estimate Permeability

In [ ]:
import matplotlib.ticker as ticker

# Estimate permeability
Swirr = 1e-3 / phit**.9
perm = coates_permeability(phit, Swirr, a=.2)
perm_kc = kozeny_carman_permeability(phit, S=1e-3)
perm_t = timur_permeability(phit, Swirr)
perm_tx = tixier_permeability(phit, Swirr)

# Plotting
perm_min_depth = 3830
perm_max_depth = 3980
fig, ax = plt.subplots(figsize=(20, 2))
ax.scatter(well_data.DEPTH, well_data.CPERM, marker='X', color='red', label='CPERM')
ax.plot(well_data.DEPTH, perm, label='Coates')
ax.plot(well_data.DEPTH, perm_kc, ':', label='KC')
ax.plot(well_data.DEPTH, perm_t, '.-', label='Timur')
ax.plot(well_data.DEPTH, perm_tx, '--', label='Tixier',)
ax.set_frame_on(False)
ax.set_yscale('log')
ax.set_ylim(1e-2, 1e5)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(
    lambda x, pos: ('{{:.{:1d}f}}'.format(int(np.maximum(-np.log10(x), 0)))).format(x)))
ax.set_xlim(perm_min_depth, perm_max_depth)
ax.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

In [ ]:
from quick_pp.core_calibration import *

poroperm_xplot(well_data.CPORE, well_data.CPERM)

## Estimate Water Saturation

In [ ]:
well_data.columns

In [ ]:
from quick_pp.saturation import *
from quick_pp.porosity import *

# Debug water saturation
water_salinity = 3.7e4
m = 1.9

phit_clay = estimate_shale_porosity(nphihc, phit)
# _, rhob_trend = min_max_line(well_data.RHOB)
# phit_clay = estimate_shale_porosity_trend(rhob_trend)

temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'metric')
rw_archie = estimate_rw_archie(phit, well_data['RT'], m=m)
rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
rw_from_surface = estimate_rw_surface(temp_grad, 0.07, 20)
rw_shale = estimate_rw_from_shale_trend(well_data.RT, phit_clay, vcld, well_data.DEPTH)
rt_shale = estimate_rt_shale(well_data.RT, vclay)

b_archie = estimate_b_waxman_smits(temp_grad, rw_archie)
b = estimate_b_waxman_smits(temp_grad, rw)

qv = estimate_qv(vcld, phit, cec_clay=.1)
qvn = estimate_qvn(vclay, phit, phit_clay)

swt_nws = normalized_waxman_smits_saturation(well_data.RT, rw, phit, vclay, phit_clay, rt_shale, m=m)
swt_archie = archie_saturation(well_data.RT, rw, phit, m=m)
swt_ws = waxman_smits_saturation(well_data['RT'], rw, phit, Qv=qvn, B=b, m=m)
sw_connectivity = connectivity_saturation(well_data['RT'], rw, phit, mu=1.6, chi_w=0.02).clip(0, 2.0)

swb = .005
rwb = .005
sw_dual = dual_water_saturation(well_data['RT'], rw, phit, 1, m, 2, swb, rwb)

# Plotting
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, sharex=True, figsize=(20, 7))

ax1.plot(well_data.DEPTH, qv, label='Qv')
ax1.plot(well_data.DEPTH, qvn, label='Qvn')
ax1.set_ylim(-.1, 10)
ax1.legend()

ax2.plot(well_data.DEPTH, b, label='b')
ax2.plot(well_data.DEPTH, b_archie, label='b_archie')
ax2.legend()

ax3.plot(well_data.DEPTH, well_data['RT'], label='RT')
ax3.plot(well_data.DEPTH, rw, label='rw')
ax3.plot(well_data.DEPTH, rw_shale, label='rw_shale')
ax3.plot(well_data.DEPTH, rt_shale, label='RT shale')
ax3.plot(well_data.DEPTH, rw_from_surface, label='rw_from_surface')
ax3.set_yscale('log')
ax3.legend()

# ax4.plot(well_data.DEPTH, qvn, 'r:', label='Qvn')
ax4.plot(well_data.DEPTH, swt_ws, label='SWT WS')
ax4.plot(well_data.DEPTH, swt_nws, label='SWT Normzalied WS')
# ax4.plot(well_data.DEPTH, sw_connectivity, label='SW_connectivity')
# ax4.plot(well_data.DEPTH, sw_dual, label='SW_dual_water_model')
ax4.plot(well_data.DEPTH, swt_archie, label='SWT Archie', linestyle='--')
ax4.plot(well_data.DEPTH, np.ones(len(sw_dual)), 'black', linestyle='--')
ax4.set_ylim(-.1, 2)
ax4.legend()

# ax4.set_xlim(3700, 4200)
fig.tight_layout()

In [ ]:
from quick_pp.saturation import pickett_plot

# Pickett plot
water_zone = well_data.copy()
water_zone['PHIT'] = phit
water_zone = water_zone[(water_zone.DEPTH > 3440) & (water_zone.DEPTH < 3460)]
if not water_zone.empty:
    fig = pickett_plot(water_zone.RT, water_zone.PHIT, m=1.85, min_rw=.035)

## QAQC Result

In [ ]:
# Update data in the project
well_data['NPHI_HC'] = nphihc
well_data['RHOB_HC'] = rhobhc
well_data['RHOB_CORR'] = rhob_corr
well_data['HC_FLAG'] = hc_flag
well_data['VSAND'] = vsand
well_data['VCLAY'] = vclay
well_data['PHIT'] = phit
well_data['PHIE'] = phie
well_data['PHID'] = phid
well_data['RW'] = rw
well_data['B'] = b
well_data['Qv'] = qv
well_data['M'] = args['sw_m']
well_data['SWT'] = swt_nws.clip(0, 1)
# well_data['SWE'] = swe
well_data['BVW'] = swt_nws.clip(0, 1) * phie
well_data['PERM'] = perm
# well_data['CPERM'] = perm
well_data['ROCK_FLAG'] = rock_flag
well_data['VHC'] = (phit * (1 - swt_nws)).clip(0, 1)
well_data['BADHOLE'] = badhole_flag
well_data['VSH_GR'] = vsh_gr

In [ ]:
from quick_pp.porosity import *

phit_shale = estimate_shale_porosity(well_data.NPHI_HC, well_data.PHIT)

plt.figure(figsize=(20, 3))
plt.plot(well_data.DEPTH, phit_shale, label='PHIT Shale')
plt.plot(well_data.DEPTH, phit_clay, label='PHIT Clay')
plt.plot(well_data.DEPTH, well_data.PHIT, label='PHIT')
plt.plot(well_data.DEPTH, well_data.NPHI, label='NPHI')
plt.plot(well_data.DEPTH, well_data.NPHI_HC, label='NPHI_HC')
plt.ylim(-.2, 1)
plt.legend()

In [ ]:
from quick_pp.saturation import *

min_depth = 4050
max_depth = 4060

copy_df = well_data[(well_data.DEPTH > min_depth) & (well_data.DEPTH < max_depth)].copy()

qv_phit_xplot(copy_df.PHIT, copy_df.Qv)

In [ ]:
from quick_pp.saturation import *

min_depth = 3900
max_depth = 4130

copy_df = well_data[(well_data.DEPTH > min_depth) & (well_data.DEPTH < max_depth)].reset_index(drop=True)
fig = rt_phit_xplot(copy_df.RT, copy_df.PHIT, m=2.6, rw=.01)

In [ ]:
from quick_pp.saturation import *
from quick_pp.porosity import *

min_depth = 3900
max_depth = 4130

copy_df = well_data[(well_data.DEPTH > min_depth) & (well_data.DEPTH < max_depth)].reset_index(drop=True)
phit_clay = estimate_shale_porosity(copy_df.NPHI_HC, copy_df.PHIT)
qvn = estimate_qvn(copy_df.VCLAY, copy_df.PHIT, phit_clay)
fig = cwa_qvn_xplot(copy_df.RT, copy_df.PHIT, qvn, m=1.9, rw=.02, slope=80)

In [ ]:
# Estimate reservoir summary
ressum_df = calc_reservoir_summary(well_data.DEPTH, vclay, phit, swt_nws, perm, zones, cutoffs=args['ressum_cutoffs'])

In [ ]:
from quick_pp.qaqc import quick_qc

test, summary_df, dist_fig, depth_fig = quick_qc(well_data, return_fig=True)

dist_fig.show()
depth_fig.show()

## Final Result

In [ ]:
# Plot the results
from quick_pp.machine_learning.feature_engineering import tight_streak_flagging
well_data['TIGHT_FLAG'] = tight_streak_flagging(well_data.RHOB, well_data.RT)
print(well_data.TIGHT_FLAG.value_counts())
# well_data.drop(columns=['VOIL', 'VGAS'], errors='ignore', inplace=True)
fig = plotly_log(well_data, well_name=well_name, column_widths=[1, 1, 1, 1, 1, 1, .3, 1, 1])
fig.show(config=dict(scrollZoom=True))
# fig.write_html(rf"{well_name}_log.html", config=dict(scrollZoom=True))

In [ ]:
# Save result to database
with db_conn.get_session() as db_session:
    project = Project(db_session, name=project_name)
    project.update_data(well_data, well_configs={well_name: args})
    project.save()